## Enviorment

In [90]:
!whoami

michael_zl_prime


In [91]:
!gcloud auth list

                  Credentialed Accounts
ACTIVE  ACCOUNT
*       312164605303-compute@developer.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [92]:
#test which virtualenv running in
import sys
sys.prefix

'/usr'

In [ ]:
#start tensor board
!/usr/local/bin/tensorboard serve --logdir gs://dl_training_results/tensorboard --port 8889 --bind_all --purge_orphaned_data True

TensorBoard 2.1.0a20191124 at http://instance-1:8889/ (Press CTRL+C to quit)
E1129 11:21:48.206579 140479723202304 _internal.py:122] ::ffff:79.181.28.67 - - [29/Nov/2019 11:21:48] code 400, message Bad request syntax ('\x16\x03\x01\x02\x00\x01\x00\x01ü\x03\x03íúh\'\x9bèWq¬I\x94Í\x85\x01@\x0cß\x85\x86\x15\x15AJñ?X7\xa0Åb+£ ¨Á\x989WUA-£¤¯£\x97\x18\x87ÍD\x93\x96Ü<Ý!¥xòÈÉW\x07ã/\x00"\x9a\x9a\x13\x01\x13\x02\x13\x03À+À/À,À0Ì©Ì¨À\x13À\x14\x00\x9c\x00\x9d\x00/\x005\x00')
E1129 11:21:49.409137 140479723202304 _internal.py:122] ::ffff:79.181.28.67 - - [29/Nov/2019 11:21:49] code 400, message Bad HTTP/0.9 request type ('\x16\x03\x01\x02\x00\x01\x00\x01ü\x03\x03¾»\x05\x9a\x94\x9e§m\x83Y7\x84\xad$teì\\æãHÎ|çq/\x8b\\¸%sQ')


---

In [93]:
import ipywidgets as widgets

In [1]:
import tensorflow as tf
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
from google.cloud import storage

from config import TRANSFORMED_TRAIN_ANNOTATIONS_PATH,TRANSFORMED_VALIDATION_ANNOTATIONS_PATH,IMAGE_SIZE
from models.six_stage_linear_model import ModelMaker
import dataset_functions
import visualizations as v

In [95]:
tf.version.VERSION

'2.1.0-dev20191124'

In [96]:
#Training config, can be moved later to main config
CACHE=False
CACHE_RAMFS=False #uses a ramfs file to force using main memory
BATCH_SIZE=32  #must be small if caching
SHUFFLE=True
PREFETCH=10  #size of prefetch size, 0 to disable

USING_TPU=False

In [ ]:
if USING_TPU:
    tpu_ip='10.0.3.2'
    # !!!MAKE SURE THE TPU ADDRESS IS CORRECT!!
    # 1.ip must be correct
    # 2.tpu must be turned on!
    # 3.version must be 'nightly-2.x'
    # 4.tpu must be reachable (check with gce netowrking/connectivity test)
    # if not this will hang
    tpu_address = 'grpc://'+tpu_ip+'8470'
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.experimental.TPUStrategy(resolver)

---
Caching using ramfs (irrelevant for tpu)

In [98]:
if CACHE_RAMFS:
    !mkdir /tmp/ramdisk
    !sudo umount /tmp/ramdisk
    !sudo mount -t ramfs -o size=512m ramfs /tmp/ramdisk
    !sudo chown $LOGNAME:$LOGNAME /tmp/ramdisk

In [99]:
if CACHE_RAMFS:
    cache_loc="/tmp/ramdisk/cache_t"
    cache_v_loc="/tmp/ramdisk/cache_v"
else:
    cache_loc=None
    cache_v_loc=None

In [100]:
cache_loc

---
# Make dataset

In [101]:
label_transformer=dataset_functions.LabelTransformer()
@tf.function
def make_label_tensors(elem):
    """Transforms a dict data element:
    1.Read jpg to tensor 
    1.1 Resize img to correct size for network
    2.Convert keypoints to correct form label tensor
    3.Convert joints to correct form label tensor
    outputs a tuple data element"""
    
    idd=elem['id']
    kpt_tr=label_transformer.keypoints_spots_vmapfn(elem['kpts'])
    paf_tr=label_transformer.joints_PAFs(elem['joints'])
    
    image_raw=elem["image_raw"]
    image=tf.image.decode_jpeg(image_raw,channels=3)
    image=tf.image.convert_image_dtype(image,dtype=tf.float32)
    image=tf.image.resize(image,IMAGE_SIZE)
    return image,(paf_tr,kpt_tr),idd

In [102]:
@tf.function
def place_training_labels(image,labels,idd):
    """Disterbutes labels into the correct configuration for the model, ie 4 PAF stage, 2 kpt stages
    must match the model"""
    paf_tr=labels[0]
    kpt_tr=labels[1]
    return image,(paf_tr,paf_tr,paf_tr,paf_tr,kpt_tr,kpt_tr) #this should match the model outputs, and is different for each model

Read and Parse the TFrecords

In [103]:
DATASET_SIZE=56000 #exact size not critical
DATASET_VAL_SIZE=2500 

----
figure out GCS storage


In [104]:
train_prefix=TRANSFORMED_TRAIN_ANNOTATIONS_PATH.split(os.sep)[-1]
val_prefix=TRANSFORMED_VALIDATION_ANNOTATIONS_PATH.split(os.sep)[-1]

In [105]:
bucket_name="datasets_bucket_a"
gs_prefix="gs://"

In [106]:
storage_client = storage.Client() #must have apropriate authenitication to work 

train_blobs = storage_client.list_blobs(bucket_name,prefix=train_prefix)
val_blobs = storage_client.list_blobs(bucket_name,prefix=val_prefix)

In [107]:
tfrecord_files_train=[gs_prefix+bucket_name+'/'+blob.name for blob in train_blobs]
tfrecord_files_val=[gs_prefix+bucket_name+'/'+blob.name for blob in val_blobs]

In [108]:
#tfrecord_files_train

In [109]:
ds = tf.data.TFRecordDataset(tfrecord_files_train)

In [110]:
TF_parser=dataset_functions.TFrecordParser() #used for 

#order of transformations is critical!

#TFrecord files to raw format
ds = tf.data.TFRecordDataset(tfrecord_files_train) #numf reads can be put here, but I don't think I/O is the bottleneck

#raw format to imgs,tensors(coords kpts)
ds=ds.map(TF_parser.read_tfrecord)

#cache  ,caching is here before decompressing jpgs and label tensors (should be ~9GB) , (full dataset should be ~90, cache later if RAM aviable)
if CACHE: ds=ds.cache(cache_loc)
if SHUFFLE: ds=ds.shuffle(100)    
    
#Augmentation should be here, to operate on smaller tensors
    
#imgs,tensors to label_tensors (46,46,17/38)
ds=ds.map(make_label_tensors)
#imgs,label_tensors arrange for model outputs
ds=ds.map(place_training_labels) 

#batch
ds=ds.batch(BATCH_SIZE)
#repeat
ds=ds.repeat()
#prefetch
if PREFETCH: ds=ds.prefetch(PREFETCH)

Make validation dataset

In [111]:
TF_parser=dataset_functions.TFrecordParser() #used for 

#order of transformations is critical!

#TFrecord files to raw format
ds_v = tf.data.TFRecordDataset(tfrecord_files_val) #numf reads can be put here, but I don't think I/O is the bottleneck
#raw format to imgs,tensors(coords kpts)
ds_v=ds_v.map(TF_parser.read_tfrecord)   

#cache  
if CACHE: ds_v=ds_v.cache(cache_v_loc)
    
#imgs,tensors to label_tensors (46,46,17/38)
ds_v=ds_v.map(make_label_tensors)
#imgs,label_tensors arrange for model outputs
ds_v=ds_v.map(place_training_labels) 
#batch
ds_v=ds_v.batch(BATCH_SIZE)


Examine datasets

In [112]:
st=next(iter(ds))
#st
#st_v=next(iter(ds_v))
#v.show_pafs_kpts_img()

In [113]:
len(st[0])

32

In [114]:
ds

<PrefetchDataset shapes: ((None, 368, 368, 3), ((None, 46, 46, 38), (None, 46, 46, 38), (None, 46, 46, 38), (None, 46, 46, 38), (None, 46, 46, 17), (None, 46, 46, 17))), types: (tf.float32, (tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32))>

In [115]:
ds_v

<BatchDataset shapes: ((None, 368, 368, 3), ((None, 46, 46, 38), (None, 46, 46, 38), (None, 46, 46, 38), (None, 46, 46, 38), (None, 46, 46, 17), (None, 46, 46, 17))), types: (tf.float32, (tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32))>

In [116]:
#st[0]

---
# Callbacks
**Checkpoints**

In [137]:
#local dir for CPU/GPU training
#checkpoints_dir='./checkpoints'
#!mkdir checkpoints
#gcs dir for TPU training
checkpoints_dir='gs://dl_training_results/checkpoints'
#make sure the directory exists, if not make it.

In [138]:
import datetime
now=datetime.datetime.now().strftime("%a%d%m%y-%H%M")


In [139]:
t=datetime.datetime.now()
now=t.strftime("%a%d%m%y-%H%M")
checkpoints_path = checkpoints_dir+"/ModelWeights-"+datetime.datetime.now().strftime("%a%d%m%y-%H%M")+"-{epoch:04d}.ckpt" 
checkpoints_path

'gs://dl_training_results/checkpoints/ModelWeights-Wed271119-1403-{epoch:04d}.ckpt'

In [140]:
#if this fails, the checkpointing won't work
!touch /tmp/test
!gsutil cp /tmp/test {checkpoints_dir}/test
!gsutil rm {checkpoints_dir}/test

Copying file:///tmp/test [Content-Type=application/octet-stream]...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              
Removing gs://dl_training_results/checkpoints/test...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              


In [141]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoints_path,
                                             save_weights_only=True,
                                             verbose=1)

**Logging**

In [142]:
logging_dir='gs://dl_training_results/logging/'
csv_logger = tf.keras.callbacks.CSVLogger(logging_dir)


In [143]:
#if this fails, the checkpointing won't work
!touch /tmp/test
!gsutil cp /tmp/test {logging_dir}/test
!gsutil rm {logging_dir}/test

Copying file:///tmp/test [Content-Type=application/octet-stream]...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              
Removing gs://dl_training_results/logging//test...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              


**TensorBoard**

In [144]:
tensorboard_dir='gs://dl_training_results/tensorboard/'+datetime.datetime.now().strftime("%a%d%m%y-%H%M")
tensorboard_callback=tf.keras.callbacks.TensorBoard(
    log_dir=tensorboard_dir
    ,update_freq=5000 #to update sooner than every epoch
)

In [145]:
#if this fails, the checkpointing won't work
!touch /tmp/test
!gsutil cp /tmp/test {tensorboard_dir}/test
!gsutil rm {tensorboard_dir}/test

Copying file:///tmp/test [Content-Type=application/octet-stream]...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              
Removing gs://dl_training_results/tensorboard/Wed271119-1403/test...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              


In [146]:
#the command to run tensorboard
#tensorboard serve --logdir gs://dl_training_results/tensorboard --port 8889 --bind_all

---
# Model

In [147]:
@tf.function
def mse_2d_loss(y_true, y_pred):
    pixel_losses=tf.keras.losses.mean_squared_error(y_true, y_pred)
    return tf.math.reduce_mean(pixel_losses,axis=-1)

In [148]:
class AnalogRecall(tf.keras.metrics.Metric):
    """This metric returns the overlap of the true gaussian 'islands' and the predicted ones"""
    def __init__(self, name='analog_recall_error',thershold=0.01, **kwargs):
        super(AnalogRecall, self).__init__(name=name, **kwargs)
        self.mean = self.add_weight(name='mean', initializer='zeros')
        self.thershold=thershold
    
    
    def update_state(self, y_true, y_pred,**kwargs):     
        
        true_island_sum=tf.reduce_sum(tf.where(y_true>self.thershold,y_true,0)) #get sum of the true island
        true_island_size=tf.cast(tf.math.count_nonzero(y_true>self.thershold),dtype=tf.float32) #get size of the island   
        mean_island_true=true_island_sum/true_island_size #average island value
               
        err=y_true-y_pred  #get all error
        recall_err=tf.where(err>0,err,0)  #get only recall error, the parts where prediction is missing
        recall_err_sum=tf.reduce_sum(recall_err)        
        err_island_size=tf.cast(tf.math.count_nonzero(y_true>self.thershold),tf.float32)  #get size of the islands above thershold
        
        mean_island_recall_err=recall_err_sum/err_island_size  #mean of the error 
        
        value=1-mean_island_recall_err/mean_island_true #the 1- converts it to recall accuracy onstead of err        
        self.mean.assign_add(value)
        
    def result(self):
        return self.mean

In [149]:
def define():
    model_maker=ModelMaker()
    train_model,test_model=model_maker.create_models()
    
    #this must match the model output order
    metrics={'stage1paf_output':  [AnalogRecall(),tf.keras.metrics.MeanAbsoluteError()]
         ,'stage2paf_output':  [AnalogRecall(),tf.keras.metrics.MeanAbsoluteError()]
         ,'stage3paf_output':  [AnalogRecall(),tf.keras.metrics.MeanAbsoluteError()]
         ,'stage4paf_output':  [AnalogRecall(),tf.keras.metrics.MeanAbsoluteError()]
         ,'stage5heatmap_output': [AnalogRecall(),tf.keras.metrics.MeanAbsoluteError()]    
         ,'stage6heatmap_output': [AnalogRecall(),tf.keras.metrics.MeanAbsoluteError()]
        }
    
    train_model.compile(optimizer=tf.keras.optimizers.Adam()
                    ,loss=mse_2d_loss
                    ,metrics=metrics                    
                   )

if USING_TPU:
    with strategy.scope():
        define()
else:
    define()

## Load weights

In [150]:
#load latest weights
latest = tf.train.latest_checkpoint(checkpoints_dir)
if latest:
    print("Found checkpoint: %s" % latest)

Found checkpoint: gs://dl_training_results/checkpoints/ModelWeights-Wed271119-1356-0002.ckpt


In [151]:
load=widgets.Checkbox(
    value=False,
    description='Load last weights'
)
load

Checkbox(value=False, description='Load last weights')

In [152]:
if latest and (load.value):
    train_model.load_weights(latest)

---
Actually training

In [153]:
steps_per_epoch=int(DATASET_SIZE/BATCH_SIZE)

In [154]:
train_model.fit(
    ds,epochs=2
    ,steps_per_epoch=7
    ,validation_data=ds_v
    ,callbacks=[cp_callback
                #,csv_logger
                ,tensorboard_callback]
)

Train for 7 steps
Epoch 1/2


1/7 [===>..........................] - ETA: 6:21 - loss: 0.2048 - stage1paf_output_loss: 0.0429 - stage2paf_output_loss: 0.0260 - stage3paf_output_loss: 0.0169 - stage4paf_output_loss: 0.0065 - stage5heatmap_output_loss: 0.0427 - stage6heatmap_output_loss: 0.0697 - stage1paf_output_analog_recall_error: 1.8624 - stage1paf_output_mean_absolute_error: 0.1273 - stage2paf_output_analog_recall_error: 1.5509 - stage2paf_output_mean_absolute_error: 0.0916 - stage3paf_output_analog_recall_error: 1.1052 - stage3paf_output_mean_absolute_error: 0.0634 - stage4paf_output_analog_recall_error: 1.3757 - stage4paf_output_mean_absolute_error: 0.0422 - stage5heatmap_output_analog_recall_error: 1.4097 - stage5heatmap_output_mean_absolute_error: 0.1144 - stage6heatmap_output_analog_recall_error: 2.6775 - stage6heatmap_output_mean_absolute_error: 0.1736WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (1.591561). Check your callbacks.


2/7 [=======>......................] - ETA: 2:45 - loss: 0.1926 - stage1paf_output_loss: 0.0256 - stage2paf_output_loss: 0.0313 - stage3paf_output_loss: 0.0240 - stage4paf_output_loss: 0.0052 - stage5heatmap_output_loss: 0.0446 - stage6heatmap_output_loss: 0.0618 - stage1paf_output_analog_recall_error: 4.0064 - stage1paf_output_mean_absolute_error: 0.0872 - stage2paf_output_analog_recall_error: 2.0914 - stage2paf_output_mean_absolute_error: 0.0742 - stage3paf_output_analog_recall_error: 3.4491 - stage3paf_output_mean_absolute_error: 0.0723 - stage4paf_output_analog_recall_error: 1.7662 - stage4paf_output_mean_absolute_error: 0.0286 - stage5heatmap_output_analog_recall_error: 2.3495 - stage5heatmap_output_mean_absolute_error: 0.1092 - stage6heatmap_output_analog_recall_error: 3.8450 - stage6heatmap_output_mean_absolute_error: 0.1464WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.792061). Check your callbacks.


3/7 [===========>..................] - ETA: 1:29 - loss: 0.1696 - stage1paf_output_loss: 0.0176 - stage2paf_output_loss: 0.0213 - stage3paf_output_loss: 0.0164 - stage4paf_output_loss: 0.0042 - stage5heatmap_output_loss: 0.0437 - stage6heatmap_output_loss: 0.0665 - stage1paf_output_analog_recall_error: 4.2879 - stage1paf_output_mean_absolute_error: 0.0619 - stage2paf_output_analog_recall_error: 2.1810 - stage2paf_output_mean_absolute_error: 0.0513 - stage3paf_output_analog_recall_error: 3.5646 - stage3paf_output_mean_absolute_error: 0.0501 - stage4paf_output_analog_recall_error: 2.1133 - stage4paf_output_mean_absolute_error: 0.0242 - stage5heatmap_output_analog_recall_error: 2.4708 - stage5heatmap_output_mean_absolute_error: 0.0990 - stage6heatmap_output_analog_recall_error: 5.3018 - stage6heatmap_output_mean_absolute_error: 0.1467WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.407508). Check your callbacks.


6/7 [========================>.....] - ETA: 11s - loss: 0.1295 - stage1paf_output_loss: 0.0092 - stage2paf_output_loss: 0.0110 - stage3paf_output_loss: 0.0086 - stage4paf_output_loss: 0.0025 - stage5heatmap_output_loss: 0.0430 - stage6heatmap_output_loss: 0.0551 - stage1paf_output_analog_recall_error: 4.3614 - stage1paf_output_mean_absolute_error: 0.0322 - stage2paf_output_analog_recall_error: 2.2232 - stage2paf_output_mean_absolute_error: 0.0266 - stage3paf_output_analog_recall_error: 3.5862 - stage3paf_output_mean_absolute_error: 0.0260 - stage4paf_output_analog_recall_error: 2.1946 - stage4paf_output_mean_absolute_error: 0.0131 - stage5heatmap_output_analog_recall_error: 3.7756 - stage5heatmap_output_mean_absolute_error: 0.0938 - stage6heatmap_output_analog_recall_error: 5.2142 - stage6heatmap_output_mean_absolute_error: 0.1126WARNING:tensorflow:5 out of the last 13 calls to <function mse_2d_loss at 0x7ff2058f9620> triggered tf.function retracing. Tracing is expensive and the excess


Epoch 00001: saving model to gs://dl_training_results/checkpoints/ModelWeights-Wed271119-1403-0001.ckpt
7/7 [==============================] - 128s 18s/step - loss: 0.1236 - stage1paf_output_loss: 0.0080 - stage2paf_output_loss: 0.0096 - stage3paf_output_loss: 0.0075 - stage4paf_output_loss: 0.0023 - stage5heatmap_output_loss: 0.0428 - stage6heatmap_output_loss: 0.0535 - stage1paf_output_analog_recall_error: 4.3630 - stage1paf_output_mean_absolute_error: 0.0278 - stage2paf_output_analog_recall_error: 2.2510 - stage2paf_output_mean_absolute_error: 0.0230 - stage3paf_output_analog_recall_error: 3.5925 - stage3paf_output_mean_absolute_error: 0.0225 - stage4paf_output_analog_recall_error: 2.2114 - stage4paf_output_mean_absolute_error: 0.0115 - stage5heatmap_output_analog_recall_error: 4.3779 - stage5heatmap_output_mean_absolute_error: 0.0934 - stage6heatmap_output_analog_recall_error: 5.2040 - stage6heatmap_output_mean_absolute_error: 0.1080 - val_loss: 0.0847 - val_stage1paf_output_loss:

1/7 [===>..........................] - ETA: 8s - loss: 0.0987 - stage1paf_output_loss: 7.1053e-04 - stage2paf_output_loss: 7.1072e-04 - stage3paf_output_loss: 7.1035e-04 - stage4paf_output_loss: 7.1048e-04 - stage5heatmap_output_loss: 0.0467 - stage6heatmap_output_loss: 0.0492 - stage1paf_output_analog_recall_error: -0.0176 - stage1paf_output_mean_absolute_error: 0.0014 - stage2paf_output_analog_recall_error: -0.0128 - stage2paf_output_mean_absolute_error: 0.0014 - stage3paf_output_analog_recall_error: -0.0154 - stage3paf_output_mean_absolute_error: 0.0014 - stage4paf_output_analog_recall_error: -0.0086 - stage4paf_output_mean_absolute_error: 0.0014 - stage5heatmap_output_analog_recall_error: 0.9052 - stage5heatmap_output_mean_absolute_error: 0.1069 - stage6heatmap_output_analog_recall_error: 0.0422 - stage6heatmap_output_mean_absolute_error: 0.0885WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.358219). Check your callbacks.


6/7 [========================>.....] - ETA: 0s - loss: 0.0876 - stage1paf_output_loss: 7.5189e-04 - stage2paf_output_loss: 7.5281e-04 - stage3paf_output_loss: 7.5178e-04 - stage4paf_output_loss: 7.5175e-04 - stage5heatmap_output_loss: 0.0414 - stage6heatmap_output_loss: 0.0432 - stage1paf_output_analog_recall_error: -0.0671 - stage1paf_output_mean_absolute_error: 0.0014 - stage2paf_output_analog_recall_error: 0.0024 - stage2paf_output_mean_absolute_error: 0.0015 - stage3paf_output_analog_recall_error: -0.0648 - stage3paf_output_mean_absolute_error: 0.0014 - stage4paf_output_analog_recall_error: -0.0466 - stage4paf_output_mean_absolute_error: 0.0014 - stage5heatmap_output_analog_recall_error: 3.6529 - stage5heatmap_output_mean_absolute_error: 0.0915 - stage6heatmap_output_analog_recall_error: 0.2867 - stage6heatmap_output_mean_absolute_error: 0.0808
Epoch 00002: saving model to gs://dl_training_results/checkpoints/ModelWeights-Wed271119-1403-0002.ckpt
7/7 [==============================

In [155]:
#v.show_pafs_kpts_img(img.numpy(),paf.numpy(),kpt.numpy(),1,1) #can be used to draw the tensor data

# !!!!!!!IT IS FUCKING WORKING!!!!!!!